In [1]:
import io
import sys
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize                  # flatten json to dataframe
from collections import OrderedDict
from simple_salesforce import Salesforce                   # !pip3 install salesforce-reporting
from pardot.client import APIClient                        # !pip3 install pardot-api-client
import pytz
import datetime
import re
from uszipcode import SearchEngine                         # !pip3 install uszipcode
search = SearchEngine(simple_zipcode=False)                # set simple_zipcode=False to use rich info database
                                                           # !pip3 install --upgrade uszipcode # upgrade databases
from simple_salesforce import Salesforce                   # !pip3 install simple-salesforce
from salesforce_reporting import Connection, ReportParser  
with open('.sfdc') as f:
    uname, spass, stoken, ppass, ptoken = [x.strip("\n") for x in f.readlines()]
sf = Salesforce(username=uname, password=spass, security_token=stoken)          # authenticate
end = datetime.datetime.now(pytz.UTC)                                           # salesforce API requires UTC
search = SearchEngine(simple_zipcode=True)                # False uses rich info database
print("@ ", end)

Start downloading data for rich info zipcode database, total size 450+MB ...
  10 MB finished ...
  20 MB finished ...
  30 MB finished ...
  40 MB finished ...
  50 MB finished ...
  60 MB finished ...
  70 MB finished ...
  80 MB finished ...
  90 MB finished ...
  100 MB finished ...
  110 MB finished ...
  120 MB finished ...
  130 MB finished ...
  140 MB finished ...
  150 MB finished ...
  160 MB finished ...
  170 MB finished ...
  180 MB finished ...
  190 MB finished ...
  200 MB finished ...
  210 MB finished ...
  220 MB finished ...
  230 MB finished ...
  240 MB finished ...
  250 MB finished ...
  260 MB finished ...
  270 MB finished ...
  280 MB finished ...
  290 MB finished ...
  300 MB finished ...
  310 MB finished ...
  320 MB finished ...
  330 MB finished ...
  340 MB finished ...
  350 MB finished ...
  360 MB finished ...
  370 MB finished ...
  380 MB finished ...
  390 MB finished ...
  400 MB finished ...
  410 MB finished ...
  420 MB finished ...
  430 MB

### pull Target Account List (TAL) from Salesforce and flatten to dataframe

In [2]:
tal_soql = ("SELECT BillingAddress, CAPDB__c, Child_Account__c, D_B_List__c, Digital_Workplace_GF__c, Digital_Workplace_WS__c, domain__c, DOrg_List__c, DSCORGPKG__DiscoverOrg_ID__c, DunsNumber, Employee_Communications_GF__c, Employee_Communications_WS__c, Fortune_1000__c, FSH_List__c, FWI_Cloud_Author_Licenses__c, FWI_Cloud_Contributor_Licenses__c, FWI_Cloud_Viewer_Licenses__c, g_slots__c, geoloc__c, Id, Industry_Vertical__c, JigsawCompanyId, LID__LinkedIn_Company_Id__c, LinkedIn_Profile_URL__c, Meetings_and_Events_External_GF__c, Meetings_and_Events_External_WS__c, MSA__c, NAICS_2_Digit_Desc__c, NAICS_3_Digit_Desc__c, NaicsCode, NaicsDesc, Name, NCAA_Conference__c, NCAA_Division__c, Number_of_Employees__c, NumberOfEmployees, of_Active_Opps__c, of_Opportunities__c, of_Opps_Created_Last_30_Days__c, of_Opps_Created_this_Calendar_Year__c, OwnerId, Parent_Account__c, Parent_Account_CAPDB_Score__c, ParentId, Phone, ShippingAddress, Sic, SicDesc, Target__c, TickerSymbol, Top_Level_Account_Customer_Type__c, Top_Level_Parent__c, Top_Level_Parent_ID__c, Top_Level_Zip__c, Total_Annual_Recurring_Revenue__c, Total_Net_Software_Pro_Serv__c, Total_Net_Software_this_Calendar_Year__c, Website FROM ACCOUNT WHERE CAPDB__c IN ('A', 'B', 'C', 'D') LIMIT 12")

tal_query = sf.query(tal_soql)                                       # salesforce query

tal_df = json_normalize(tal_query['records'])                        # flatten json response to dataframe

# tal_df.to_csv (r'~/Desktop/tal.csv', index = None, header=True)    # save to csv to verify
print("@ ", end)

SalesforceMalformedRequest: Malformed request https://fwi.my.salesforce.com/services/data/v38.0/query/?q=SELECT+BillingAddress%2C+CAPDB__c%2C+Child_Account__c%2C+D_B_List__c%2C+Digital_Workplace_GF__c%2C+Digital_Workplace_WS__c%2C+domain__c%2C+DOrg_List__c%2C+DSCORGPKG__DiscoverOrg_ID__c%2C+DunsNumber%2C+Employee_Communications_GF__c%2C+Employee_Communications_WS__c%2C+Fortune_1000__c%2C+FSH_List__c%2C+FWI_Cloud_Author_Licenses__c%2C+FWI_Cloud_Contributor_Licenses__c%2C+FWI_Cloud_Viewer_Licenses__c%2C+g_slots__c%2C+geoloc__c%2C+Id%2C+Industry_Vertical__c%2C+JigsawCompanyId%2C+LID__LinkedIn_Company_Id__c%2C+LinkedIn_Profile_URL__c%2C+Meetings_and_Events_External_GF__c%2C+Meetings_and_Events_External_WS__c%2C+MSA__c%2C+NAICS_2_Digit_Desc__c%2C+NAICS_3_Digit_Desc__c%2C+NaicsCode%2C+NaicsDesc%2C+Name%2C+NCAA_Conference__c%2C+NCAA_Division__c%2C+Number_of_Employees__c%2C+NumberOfEmployees%2C+of_Active_Opps__c%2C+of_Opportunities__c%2C+of_Opps_Created_Last_30_Days__c%2C+of_Opps_Created_this_

### pull recent leads w/o TAL marks from Salesforce and flatten to dataframe

In [3]:
lead_soql = ("SELECT Address, AnnualRevenue, CAPDB__c, Company, CompanyDunsNumber, Contact__c, Contact_Form_Comments__c, Convert__c, ConvertedAccountId, ConvertedContactId, ConvertedDate, ConvertedOpportunityId, CreatedDate, Customer__c, domain__c, DSCORGPKG__Company_HQ_Address__c, DSCORGPKG__Company_HQ_Postal_Code__c, DSCORGPKG__Company_HQ_State__c, DSCORGPKG__department__c, DSCORGPKG__DiscoverOrg_Company_ID__c, DSCORGPKG__DiscoverOrg_Technologies__c, DSCORGPKG__DO_3yr_Employees_Growth__c, DSCORGPKG__DO_3yr_Sales_Growth__c, DSCORGPKG__Fortune_Rank__c, DSCORGPKG__IT_Budget__c, DSCORGPKG__IT_Employees__c, DSCORGPKG__Job_Function__c, DSCORGPKG__LinkedIn_URL__c, DSCORGPKG__Management_Level__c, DSCORGPKG__NAICS_Codes__c, DSCORGPKG__Ownership__c, DSCORGPKG__ReportsTo__c, DSCORGPKG__SIC_Codes__c, DSCORGPKG__Twitter_URL__c, Email, FirstName, FWILeadOwner__c, h_boutique__c, h_BrandPri__c, h_BrandSec__c, h_casino__c, h_class__c, h_conf__c, h_conv__c, h_golf__c, h_manage__c, h_MeetRoomSF__c, h_openDT__c, h_owner__c, h_priceClass__c, h_resort__c, h_rest__c, h_rooms__c, h_setting__c, h_ski__c, h_spa__c, Id, Industry, Industry_Vertical__c, IsConverted, IsUnreadByOwner, JigsawContactId, Lead_Type__c, Lead_Type1__c, LeadSource, LID__LinkedIn_Company_Id__c, LID__LinkedIn_Member_Token__c, LinkedIn_Profile_URL__c, MI_Event_Date__c, Mkt_Created__c, MobilePhone, MSA__c, NAICS_Code__c, Name, Number_of_Employees__c, NumberOfEmployees, Oppty_Created__c, Opty_tf__c, OwnerId, Pass_to_Partner__c, Persona__c, Phone, pi__campaign__c, pi__conversion_date__c, pi__conversion_object_name__c, pi__conversion_object_type__c, pi__created_date__c, pi__first_activity__c, pi__first_search_term__c, pi__first_search_type__c, pi__first_touch_url__c, pi__grade__c, pi__last_activity__c, pi__pardot_hard_bounced__c, pi__Pardot_Last_Scored_At__c, pi__score__c, pi__url__c, pi__utm_campaign__c, pi__utm_content__c, pi__utm_medium__c, pi__utm_source__c, pi__utm_term__c, Prospect__c, rat__c, SalesLoft_Score__c, SalesLoft1__Active_Lead__c, SalesLoft1__Most_Recent_Cadence_Name__c, SalesLoft1__Most_Recent_Cadence_Next_Step_Due_Date__c, SalesLoft1__Most_Recent_Last_Completed_Step__c, Screens__c, Source__c, SQL_Date__c, SRL_Date__c, SRL_Reason__c, StateCode__c, Status, status__c, Target__c, Territory_BDR__c, Title, Twitter__c, Unqualified_Reason__c, URL_Parameters__c, utm_campaign_first__c, utm_campaign_last__c, utm_content_first__c, utm_content_last__c, utm_medium_first__c, utm_medium_last__c, utm_source_first__c, utm_source_last__c, utm_term_first__c, utm_term_last__c, Website from Lead WHERE CreatedDate > 2017-01-01T00:00:00Z  AND CAPDB__c NOT IN ('A', 'B', 'C', 'D') LIMIT 12")

lead_query = sf.query(lead_soql)                                       # salesforce query

lead_df = json_normalize(lead_query['records'])                        # flatten json response to dataframe

lead_df.to_csv (r'~/Desktop/lead.csv', index = None, header=True)      # save to csv to verify
print("@ ", end)

@  2019-10-27 04:57:18.762182+00:00


### find lead tal ratings

In [4]:
lead_df

,Address,AnnualRevenue,CAPDB__c,Company,CompanyDunsNumber,Contact__c,Contact_Form_Comments__c,Convert__c,ConvertedAccountId,ConvertedContactId,...,Address.city,Address.country,Address.countryCode,Address.geocodeAccuracy,Address.latitude,Address.longitude,Address.postalCode,Address.state,Address.stateCode,Address.street
0,NaN,None,None,"Newman's Own, Inc.",None,None,None,True,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,None,None,Hooters Las Vegas,None,None,None,True,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,None,None,Red Door Grill,None,None,"As of 15AUG18, Dan said: Emailed and called. N...",True,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,None,None,FENOX Automotive Components,None,None,None,True,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,None,None,Pan Pacific Hotels Group,None,None,We would like a meeting summary screen from of...,False,None,None,...,None,Singapore,SG,NaN,NaN,NaN,199591,None,None,"7500A Beach Road\r\n#03-301, The Plaza"
5,NaN,None,None,St. Patrick Catholic Church,None,None,Looking for software to run a video board at o...,True,None,None,...,Urbana,United States,US,NaN,NaN,NaN,61801,Illinois,IL,﻿708 W. Main St.
6,NaN,None,None,Local Wiring Co,None,None,Hi!\r\n\r\nLooking for a digital board solutio...,True,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,None,None,Westminster Hotel,None,None,None,True,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,None,None,So Bright Advertising,None,None,Looking for cost and capabilities of FWI with ...,True,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,None,None,[not provided],None,None,None,False,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Notes

regex operations

```python
import re
df5 = df2.copy()
df5['Invoice'] = [re.findall(r'(\d{5})', s)[0] for s in df2['Ref']]
df_m5 = df1.merge(df5[['Amount', 'Invoice']], on='Invoice')
df_m4
```

file operations

```python
df = pd.read_csv('~/Desktop/file.csv')
df.to_csv (r'~/Desktop/file.csv', index = None, header=True)
with open('csvfile.csv', 'w') as csvfile: f = csv.writer(csvfile) f.writerows(items)
```

date operations ```2017-01-01T00:00:00Z```

zip-code processing

```python
df['PostalCode'] = df['PostalCode'].str[:5]       # trim off zip+4
zipcode = search.by_zipcode("06916")
zipcode.to_json()                                       # to json
zipcode.to_dict()                                       # to dict
zip = zipcode.values()                                  # to list
zipcode

def zco(x):
    return search.by_zipcode(x).state
    df['StateCode'] = df['PostalCode'].fillna(0).astype(int).apply(zco)
```

salesforce bulk upload

```python
qdfj = qdf.to_json(orient='records')
sf.bulk.Contact.insert(data)
sf.bulk.Contact.upsert(data, 'Id')
sf.bulk.Lead.update(qdfj)
```

In [5]:
import nltk

In [ ]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [2]:
nltk.download()

NameError: name 'nltk' is not defined

In [1]:
import nltk
import ssl
ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

NameError: name '_create_unverified_https_context' is not defined